<a href="https://colab.research.google.com/github/machiwao/CCTHESS1-CCTHESS2-Dev-and-Docs/blob/jessy/heat_index_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install earthengine-api geemap --quiet

In [26]:
import ee
import geemap
import pandas as pd

In [27]:
cloud_project = 'heat-index-forecasting'

try:
  ee.Initialize(project=cloud_project)
except:
  ee.Authenticate()
  ee.Initialize(project=cloud_project)

In [28]:
# Stations

roi = ee.Geometry.Point([121.0, 14.5]) # NAIA
start = '2014-01-01'
end = '2023-12-31'

In [29]:
# Load and Processing Datasets

lst = ee.ImageCollection('MODIS/006/MOD11A2').filterDate(start, end).select('LST_Day_1km').map(lambda img: img.multiply(0.02).subtract(273.15).rename('LST').copyProperties(img, ['system:time_start']))

ndvi = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(start, end).select('NDVI').map(lambda img: img.multiply(0.0001).rename('NDVI').copyProperties(img, ['system:time_start']))

landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(start, end).filterBounds(roi).map(lambda img: (img.multiply(0.0000275).add(-0.2).copyProperties(img, ['system:time_start'])))

def addIndices(img):
  nir = img.select('SR_B5')
  red = img.select('SR_B4')
  swir = img.select('SR_B6')
  blue = img.select('SR_B2')
  swir2 = img.select('SR_B7')

  ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI') # NDBI Formula

  albedo = img.expression('0.356*blue + 0.130*red + 0.373*nir + 0.085*swir + 0.072*swir2', {'blue': blue, 'red': red, 'nir': nir, 'swir': swir, 'swir2': swir2}).rename('Albedo') # Albedo Formula

  return img.addBands([ndbi, albedo])

landsat_processed = landsat.map(addIndices)

landcover = ee.ImageCollection('MODIS/006/MCD12Q1').filterDate(start, end).select('LC_Type1').map(lambda img: img.rename('LandCover').copyProperties(img, ['system:time_start']))

In [32]:
def extractValues(img):
  stats = img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=roi,
      scale=1000,
      bestEffort=True
  )
  return ee.Feature(None, stats).set('date', img.date().format('YYYY-MM-dd'))

lst_stats = ee.FeatureCollection(lst.map(extractValues))
ndvi_stats = ee.FeatureCollection(ndvi.map(extractValues))
landsat_stats = ee.FeatureCollection(landsat_processed.map(extractValues))
landcover_stats = ee.FeatureCollection(landcover.map(extractValues))

In [34]:
df_lst = geemap.ee_to_df(lst_stats)
df_ndvi = geemap.ee_to_df(ndvi_stats)
df_landsat = geemap.ee_to_df(landsat_stats)
df_lc = geemap.ee_to_df(landcover_stats)

# Merge all dataframes on "date"
df = df_lst.merge(df_ndvi, on="date", how="outer") \
           .merge(df_landsat, on="date", how="outer") \
           .merge(df_lc, on="date", how="outer")

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

csv_filename = 'LST-and-indices.csv'
df.to_csv(csv_filename, index=False)

print(df.head(10))

     LST       date    NDVI    Albedo      NDBI  QA_PIXEL  QA_RADSAT  \
0  33.41 2014-01-01  0.2963       NaN       NaN       NaN        NaN   
1    NaN 2014-01-06     NaN  0.926645 -0.237816   0.41270       -0.2   
2  35.07 2014-01-09     NaN       NaN       NaN       NaN        NaN   
3    NaN 2014-01-17  0.2313       NaN       NaN       NaN        NaN   
4    NaN 2014-01-22     NaN  0.456753 -0.070732   0.41270       -0.2   
5  32.25 2014-01-25     NaN       NaN       NaN       NaN        NaN   
6  34.79 2014-02-02  0.2323       NaN       NaN       NaN        NaN   
7    NaN 2014-02-07     NaN  0.172547 -0.004708   0.40016       -0.2   
8  35.45 2014-02-10     NaN       NaN       NaN       NaN        NaN   
9  33.27 2014-02-18  0.2319       NaN       NaN       NaN        NaN   

      SR_B1     SR_B2     SR_B3  ...  ST_ATRAN    ST_B10  ST_CDIST   ST_DRAD  \
0       NaN       NaN       NaN  ...       NaN       NaN       NaN       NaN   
1  1.037528  1.019653  0.961985  ... -0.076140 

In [35]:
from google.colab import files
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>